# About

**This is not an experimental notebook.** In here, we implement the (at best) final version(s) of the collector.

-------------------

A list of possible domains to fetch content from:

- Politics
    - [German Bundestag](https://www.bundestag.de/) (X)
    - [House of Commons](https://reshare.ukdataservice.ac.uk/854292/) (X)
- Student/School
    - [Kaggle Student Essays](https://www.kaggle.com/competitions/learning-agency-lab-automated-essay-scoring-2/data?select=train.csv) (X)
    - Take the english dataset and translate it?
- Research
    - [Arxiv](https://arxiv.org/search/advanced?advanced=&terms-0-operator=AND&terms-0-term=language+model&terms-0-field=all&classification-physics_archives=all&classification-include_cross_list=include&date-year=&date-filter_by=date_range&date-from_date=2005&date-to_date=2020&date-date_type=submitted_date&abstracts=show&size=50&order=-announced_date_first) (X)
    - [I've seen a paper recently that creates full AI written papers. Maybe I can use it.](https://www.kaggle.com/discussions/general/527817#2958636)
- News
    - [Spiegel Online](https://www.spiegel.de/) (X)
    - [CNN Articles](https://www.kaggle.com/datasets/gowrishankarp/newspaper-text-summarization-cnn-dailymail) (X)
- Blogs/Tutorials/Forums
- Law
    - [German Open Legal Data](https://de.openlegaldata.io/) (X)
    - [European Court of Human Rights Cases](https://www.kaggle.com/datasets/mathurinache/ecthrnaacl2021/data) (X)
- Philosophy
    - Gutenberg Project ([ENG](https://www.gutenberg.org/) | [GER](https://www.projekt-gutenberg.org/)) (X)
- Literature
    - Gutenberg Project ([ENG](https://www.gutenberg.org/) | [GER](https://www.projekt-gutenberg.org/)) (X)
- Blogs, Food and Lifestyle
    - [Food Blogs](https://detailed.com/food-blogs/)
    - [WebBlogs](https://www.kaggle.com/datasets/rtatman/blog-authorship-corpus?select=blogtext.csv) (ENG)
- Religion
    - [Bible](https://github.com/mxw/grmr/blob/master/src/finaltests/bible.txt) (ENG|GER) (X)
- Gaming

Interesting Languages:

- English
- German

In [1]:
class CONFIG:
    SRC_ROOT_PATH = '/home/staff_homes/kboenisc/home/prismAI/PrismAI/src'
    SRC_ROOT_PATH_COLL = '/home/staff_homes/kboenisc/home/prismAI/PrismAI/src/data_collector'
    DATA_ROOT_PATH = '/storage/corpora/prismAI/data'

In [2]:
import sys
import importlib

# So that it includes local imports. This is some next level python shit import
sys.path.insert(0, CONFIG.SRC_ROOT_PATH)
sys.path.insert(0, CONFIG.SRC_ROOT_PATH_COLL)

In [3]:
import collector
import collected_item
import collectors.bundestag_collector
import collectors.house_of_commons_collector
import collectors.student_essays_collector
import collectors.arxiv_collector
import collectors.spiegel_collector
import collectors.cnn_news_collector
import collectors.open_legal_data_collector
import collectors.euro_court_cases_collector
import collectors.religion_collector
import collectors.gutenberg_collector
import collectors.blog_corpus_collector

importlib.reload(collector)
importlib.reload(collected_item)
importlib.reload(collectors.bundestag_collector)
importlib.reload(collectors.house_of_commons_collector)
importlib.reload(collectors.student_essays_collector)
importlib.reload(collectors.arxiv_collector)
importlib.reload(collectors.spiegel_collector)
importlib.reload(collectors.cnn_news_collector)
importlib.reload(collectors.open_legal_data_collector)
importlib.reload(collectors.euro_court_cases_collector)
importlib.reload(collectors.religion_collector)
importlib.reload(collectors.gutenberg_collector)
importlib.reload(collectors.blog_corpus_collector)


<module 'collectors.blog_corpus_collector' from '/home/staff_homes/kboenisc/home/prismAI/PrismAI/src/data_collector/collectors/blog_corpus_collector.py'>

## Init the Collectors

In [4]:
import collectors.blog_corpus_collector

collection = [
    #collectors.bundestag_collector.BundestagCollector(CONFIG.DATA_ROOT_PATH),
    #collectors.house_of_commons_collector.HouseOfCommonsCollector(CONFIG.DATA_ROOT_PATH),
    #collectors.student_essays_collector.StudentEssaysCollector(CONFIG.DATA_ROOT_PATH),
    collectors.arxiv_collector.ArxivCollector(CONFIG.DATA_ROOT_PATH),
    #collectors.spiegel_collector.SpiegelCollector(CONFIG.DATA_ROOT_PATH),
    #collectors.cnn_news_collector.CNNNewsCollector(CONFIG.DATA_ROOT_PATH),
    #collectors.open_legal_data_collector.OpenLegalDataCollector(CONFIG.DATA_ROOT_PATH),
    #collectors.euro_court_cases_collector.EuroCourtCasesCollector(CONFIG.DATA_ROOT_PATH),
    #collectors.religion_collector.ReligionCollector(CONFIG.DATA_ROOT_PATH),
    #collectors.gutenberg_collector.GutenbergCollector(CONFIG.DATA_ROOT_PATH),
    #collectors.blog_corpus_collector.BlogCorpusCollector(CONFIG.DATA_ROOT_PATH)
]

In [5]:
total_items = 0

for coll in collection:
    try:
        coll.init()
        coll.collect(force=True)
        total_items += coll.get_count()
    except Exception as ex:
        print('ERROR: Current collection failed due to an error: ')
        print(ex)
        print('\n ***** Continuing with the other collectors. ***** \n')

print('\n\n ==================================================== \n\n')
print(f'All collectors finished. Total data items: {total_items}')



 ============== Collection started for ARXIV PAPERS ============== 

https://arxiv.org/pdf/2001.01001
Saved /storage/corpora/prismAI/data/arxiv_papers/raw/2001.01001.pdf
https://arxiv.org/pdf/2001.01002
Saved /storage/corpora/prismAI/data/arxiv_papers/raw/2001.01002.pdf
https://arxiv.org/pdf/2001.01003
Saved /storage/corpora/prismAI/data/arxiv_papers/raw/2001.01003.pdf
https://arxiv.org/pdf/2001.01004
Saved /storage/corpora/prismAI/data/arxiv_papers/raw/2001.01004.pdf
https://arxiv.org/pdf/2001.01005


KeyboardInterrupt: 